In [10]:
from Helper.Helper_functions import *
from Helper.ml_models import *
from torchvision.utils import save_image
from tqdm import tqdm
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import torch

import os

# Erstelle den Speicherordner, falls er nicht existiert
output_dir = "FINAL_DATEN/Bilder_Studienarbeit"
os.makedirs(output_dir, exist_ok=True)


kit_image_dir = 'KittiDaten/training/image_2'
kit_annotation_dir = 'KittiDaten/training/semantic'

cit_image_dir = 'CityscapesDaten/images'
cit_annotation_dir = 'CityscapesDaten/semantic'


df = analyse_dataset_GRAY(kit_annotation_dir)
class_distribution_violin_plot(df, output='KittiDaten')
stratified_kfold_and_violin_plot(df, output='KittiDaten',  k=5)


df2 = analyse_dataset_GRAY(cit_annotation_dir)
class_distribution_violin_plot(df2, output='CityscapesDaten')
stratified_kfold_and_violin_plot(df2, output='CityscapesDaten',  k=5)

In [11]:
def analyse_dataset_GRAY(path, classes = None):
    if classes is None:
        classes = {
            7: 'road',
            8: 'sidewalk',
            11: 'building',
            12: 'wall',
            13: 'fence',
            17: 'pole',
            19: 'traffic light',
            20: 'traffic sign',
            21: 'vegetation',
            22: 'terrain',
            23: 'sky',
            24: 'person',
            25: 'rider',
            26: 'car',
            27: 'truck',
            28: 'bus',
            31: 'train',
            32: 'motorcycle',
            33: 'bicycle',
            0: 'unlabeled'
        }

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Initialize the counts
    counts = {class_name: [] for class_name in classes.values()}      
    absolute_counts = {class_name: [] for class_name in classes.values()} 
    image_names = []
    
    def count_classes(image_path):
        # Initialize the counts for this image
        image_counts = {class_name: 0 for class_name in classes.values()}
        absolute_image_counts = {class_name: 0 for class_name in classes.values()} 
        # Open the image and convert it to grayscale
        image = Image.open(image_path).convert('L')
        # Calculate the total number of pixels for this image
        total_pixels = np.prod(image.size)
    # Convert the image to a PyTorch tensor and send it to the device
        image_tensor = torch.from_numpy(np.array(image)).to(device)
        # Iterate over each pixel in the image
        for rgb in classes.keys():
            # Create a mask for the current class
            mask = (image_tensor == torch.tensor(rgb, device=device))
            # Count the pixels in the mask and add them to the count for the current class
            absolute_image_counts[classes[rgb]] += mask.sum().item()
            image_counts[classes[rgb]] += mask.sum().item() / total_pixels
        # Add the counts for this image to the overall counts
        for class_name in classes.values():
            counts[class_name].append(image_counts[class_name])
            absolute_counts[class_name].append(absolute_image_counts[class_name])
        # Store the image name
        image_names.append(os.path.basename(image_path))

    # Get the total number of images in the directory
    total_images = len(os.listdir(path))
    
    # Iterate over all the image files in the directory
    for i, image_file in tqdm(enumerate(os.listdir(path)), total=total_images, desc='Analyzing images'):
        # Print the progress
        #print(f'Analyzing image {i}/{total_images}')
        # Count the classes in the image
        count_classes(os.path.join(path, image_file))

    # Create the DataFrame
    data = []
    for i, image_name in tqdm(enumerate(image_names), total=total_images, desc='Creating DataFrame'):
        for class_name in classes.values():
            data.append({"Image": image_name, "Class": class_name, "Pixel Percentage": counts[class_name][i], "Pixel Count": absolute_counts[class_name][i]})
    df = pd.DataFrame(data)

    return df

def analyse_dataset_RGB(path):
    classes = {
        (128, 64, 128): 'road',
        (244, 35, 232): 'sidewalk',
        (70, 70, 70): 'building',
        (102, 102, 156): 'wall',
        (190, 153, 153): 'fence',
        (153, 153, 153): 'pole',
        (250, 170, 30): 'traffic light',
        (220, 220, 0): 'traffic sign',
        (107, 142, 35): 'vegetation',
        (152, 251, 152): 'terrain',
        (70, 130, 180): 'sky',
        (220, 20, 60): 'person',
        (255, 0, 0): 'rider',
        (0, 0, 142): 'car',
        (0, 0, 70): 'truck',
        (0, 60, 100): 'bus',
        (0, 80, 100): 'train',
        (0, 0, 230): 'motorcycle',
        (119, 11, 32): 'bicycle',
        (0, 0, 0): 'unlabeled'
    }

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Initialize the counts
    counts = {class_name: [] for class_name in classes.values()}      
    image_names = []

    def count_classes(image_path):
        # Initialize the counts for this image
        image_counts = {class_name: 0 for class_name in classes.values()}
        # Open the image and convert it to RGB
        image = Image.open(image_path).convert('RGB')
        # Calculate the total number of pixels for this image
        total_pixels = np.prod(image.size)
        # Convert the image to a PyTorch tensor and send it to the device
        image_tensor = torch.from_numpy(np.array(image)).to(device)
        # Iterate over each pixel in the image
        for rgb in classes.keys():
            # Create a mask for the current class
            mask = (image_tensor == torch.tensor(rgb, device=device)).all(dim=2)
            # Count the pixels in the mask and add them to the count for the current class
            image_counts[classes[rgb]] += mask.sum().item() / total_pixels
        # Add the counts for this image to the overall counts
        for class_name in classes.values():
            counts[class_name].append(image_counts[class_name])
        # Store the image name
        image_names.append(os.path.basename(image_path))

    # Get the total number of images in the directory
    total_images = len(os.listdir(path))
    
    # Iterate over all the image files in the directory
    for i, image_file in enumerate(os.listdir(path)):
        # Print the progress
        #print(f'Analyzing image {i}/{total_images}')
        # Count the classes in the image
        count_classes(os.path.join(path, image_file))

    # Create the DataFrame
    data = []
    for i, image_name in enumerate(image_names):
        for class_name in classes.values():
            data.append({"Image": image_name, "Class": class_name, "Pixel Count": counts[class_name][i]})
    df = pd.DataFrame(data)

    return df

def class_distribution_violin_plot(df):
    # Min-Max normalisieren der "Pixel Count"-Werte für jede Klasse
    df2 = df.copy()
    df2['Pixelanteil pro Bild'] = df.groupby('Class')['Pixel Percentage'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

    # Erstellen eines horizontalen Violinplots
    plt.figure(figsize=(10, 5))
    sns.violinplot(x="Pixelanteil pro Bild", y="Class", data=df2, orient='h', cut=0, inner='quart', density_norm='count')
    plt.title('Verteilung der min-max normalisierten Pixelanzahl pro Bild für jede Klasse')
    plt.xlabel("Pixelanteil pro Bild")
    plt.ylabel("Klasse")
    plt.tight_layout()

    # Speichern des Plots
    save_path = os.path.join(output_dir, "Verteilung_Pixelanzahl.png")
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Plot gespeichert: {save_path}")

def stratified_kfold_and_violin_plot(df, k=5):
    # Die Klasse für jedes Bild abrufen
    image_classes = df.groupby('Image')['Class'].first().values

    # Initialisieren des StratifiedKFold-Objekts
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    # Eine neue Spalte zur Original-DataFrame hinzufügen, um die Quelle anzugeben
    df['Quelle'] = 'Original'

    # Bilder in k Gruppen aufteilen
    for i, (train_index, test_index) in enumerate(skf.split(df['Image'].unique(), image_classes)):
        # Bilder für diese Gruppe abrufen
        group_images = df['Image'].unique()[test_index]
        # Zeilen für diese Gruppe abrufen
        group_rows = df[df['Image'].isin(group_images)].copy()  # Kopie erstellen, um eine SettingWithCopyWarning zu vermeiden
        # Neue Spalte zur k-fold DataFrame hinzufügen, um die Quelle anzugeben
        group_rows['Quelle'] = f'Fold {i}'

        # Die Original-DataFrame und die k-fold DataFrame zusammenfügen
        combined_df = pd.concat([df, group_rows])

        # Violinplot für die kombinierten Daten erstellen
        plt.figure(figsize=(10, 5))
        sns.violinplot(x="Pixel Percentage", y="Class", hue="Quelle", split=True, data=combined_df, orient='h', cut=0, inner='quart', density_norm='count')
        plt.title(f'Vergleich der ursprünglichen Verteilung und der Verteilung für Fold {i}')
        plt.xlabel("Pixelanteil pro Bild")
        plt.ylabel("Klasse")
        plt.legend(title="Quelle")
        plt.tight_layout()

        # Speichern des Plots
        save_path = os.path.join(output_dir, f"Vergleich_Fold_{i}.png")
        plt.savefig(save_path, dpi=300)
        plt.close()
        print(f"Plot gespeichert: {save_path}")

# Analyse Kitti Dataset und Cityscapes Dataset

# 

In [12]:
# kit_image_dir = 'KittiDaten/training/image_2'
# kit_annotation_dir = 'KittiDaten/training/semantic'

city_classes = {
    0: 'road',
    1: 'sidewalk',
    2: 'building',
    3: 'wall',
    4: 'fence',
    5: 'pole',
    6: 'traffic light',
    7: 'traffic sign',
    8: 'vegetation',
    9: 'terrain',
    10: 'sky',
    11: 'person',
    12: 'rider',
    13: 'car',
    14: 'truck',
    15: 'bus',
    16: 'train',
    17: 'motorcycle',
    18: 'bicycle',
    19: 'unlabeled',
}

cit_image_dir = 'CityscapesDaten/images'
cit_annotation_dir = 'CityscapesDaten/semantic'

# df = analyse_dataset_GRAY(kit_annotation_dir)
# class_distribution_violin_plot(df)
# stratified_kfold_and_violin_plot(df, k=5)


df2 = analyse_dataset_GRAY(cit_annotation_dir, classes=city_classes)
class_distribution_violin_plot(df2)
stratified_kfold_and_violin_plot(df2,  k=5)

Creating DataFrame: 100%|██████████| 3475/3475 [00:00<00:00, 162464.82it/s]


Plot gespeichert: FINAL_DATEN/Bilder_Studienarbeit/Verteilung_Pixelanzahl.png
Plot gespeichert: FINAL_DATEN/Bilder_Studienarbeit/Vergleich_Fold_0.png
Plot gespeichert: FINAL_DATEN/Bilder_Studienarbeit/Vergleich_Fold_1.png
Plot gespeichert: FINAL_DATEN/Bilder_Studienarbeit/Vergleich_Fold_2.png
Plot gespeichert: FINAL_DATEN/Bilder_Studienarbeit/Vergleich_Fold_3.png
Plot gespeichert: FINAL_DATEN/Bilder_Studienarbeit/Vergleich_Fold_4.png


In [4]:
def calculate_class_weights(df):
    # Calculate the total pixel count for each class
    total_pixel_counts = df.groupby('Class')['Pixel Count'].sum()
    
    # Reindex the total_pixel_counts series to match the order of city_classes
    total_pixel_counts = total_pixel_counts.reindex(city_classes.values())


    # Calculate the total pixel count
    total_pixels = total_pixel_counts.sum()

    # Calculate the class weights
    class_weights = total_pixels / (20 * (total_pixel_counts))
    
    # Convert class weights to a PyTorch tensor
    class_weights_tensor = torch.from_numpy(class_weights.values.astype(np.float32))
    
    # Save the tensor to a file
    torch.save(class_weights_tensor, 'CityscapesDaten/class_weights.pt')

    return class_weights, total_pixel_counts


class_weights, total = calculate_class_weights(df2)
sorted_class_weights = class_weights.sort_values(ascending=True)
sorted_total = total.sort_values(ascending=False)

weights_equal_test = class_weights.values *  total.values
arr = np.array(weights_equal_test)
print(np.unique(arr, return_counts=True))

print(class_weights)
total_weight_sum = 0
for weight in class_weights:
    total_weight_sum += weight
print(f'Sum of all weights {total_weight_sum}')

(array([3.6438016e+08]), array([20]))
Class
road              0.152993
sidewalk          0.944824
building          0.249295
wall              8.490185
fence             6.509589
pole              4.478882
traffic light    27.426945
traffic sign      9.970298
vegetation        0.350820
terrain           5.090917
sky               1.441691
person            4.598446
rider            38.604672
car               0.816887
truck            20.783080
bus              22.012393
train            26.222695
motorcycle       58.974866
bicycle          12.408356
unlabeled         0.430046
Name: Pixel Count, dtype: float64
Sum of all weights249.95787985107148


In [5]:


class_weights_tensor = torch.load('CityscapesDaten/class_weights.pt')

# Print out the values
print(class_weights_tensor)

tensor([ 0.1530,  0.9448,  0.2493,  8.4902,  6.5096,  4.4789, 27.4269,  9.9703,
         0.3508,  5.0909,  1.4417,  4.5984, 38.6047,  0.8169, 20.7831, 22.0124,
        26.2227, 58.9749, 12.4084,  0.4300])


In [31]:
from sklearn.preprocessing import LabelEncoder

def calculate_class_weights(df, city_classes):
    # # Convert class labels to integers
    # le = LabelEncoder()
    # df['Class'] = le.fit_transform(df['Class'])

    # Compute the class weights
    print(f'{np.unique(df["Class"])} / \n {df.groupby("Class")["Pixel Count"].sum()}')
    class_weights = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(df['Class']), y=np.array(df2.groupby('Class')['Pixel Count'].sum()))

    # Create a dictionary mapping class labels to their weights
    class_weights_dict = {cls: weight for cls, weight in zip(np.unique(df['Class']), class_weights)}

    print("Unique classes in DataFrame: ", np.unique(df['Class']))
    print("Class weights dictionary: ", class_weights_dict)

    # Reorder the class weights to match the order of city_classes
    class_weights_ordered = [class_weights_dict.get(cls, 1) for cls in city_classes.keys()]

    # Convert class weights to a PyTorch tensor
    class_weights_tensor = torch.tensor(class_weights_ordered, dtype=torch.float32)

    # Save the tensor to a file
    torch.save(class_weights_tensor, 'CityscapesDaten/class_weights.pt')

    return class_weights_ordered

# Use the function
city_classes = {
    0: 'road',
    1: 'sidewalk',
    2: 'building',
    3: 'wall',
    4: 'fence',
    5: 'pole',
    6: 'traffic light',
    7: 'traffic sign',
    8: 'vegetation',
    9: 'terrain',
    10: 'sky',
    11: 'person',
    12: 'rider',
    13: 'car',
    14: 'truck',
    15: 'bus',
    16: 'train',
    17: 'motorcycle',
    18: 'bicycle',
    19: 'unlabeled',
}
class_weights = calculate_class_weights(df2, city_classes)

# Load the tensor from the file
class_weights_tensor = torch.load('CityscapesDaten/class_weights.pt')

# Print out the values
print(class_weights_tensor)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] / 
 Class
0       29365708
1     1461641548
2       16553410
3      446059503
4       55975907
5        6178567
6       79239848
7       81355164
8        9438758
9     2381680967
10     385659445
11     252744993
12      71574562
13      13285481
14      36546566
15      13895603
16      17532539
17     847304822
18    1038651996
19      42917813
Name: Pixel Count, dtype: int64


ValueError: classes should include all valid labels that can be in y

In [30]:
pixel_count_dict = np.array(df2.groupby('Class')['Pixel Count'].sum())

print(f'{pixel_count_dict} \n shape: {pixel_count_dict.shape}')

[  29365708 1461641548   16553410  446059503   55975907    6178567
   79239848   81355164    9438758 2381680967  385659445  252744993
   71574562   13285481   36546566   13895603   17532539  847304822
 1038651996   42917813] 
 shape: (20,)


In [17]:
print(np.unique(df2['Class']))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
